# System Checker - AWS 1

Der ```System Checker``` überprüft welche Versionen der für die Lehrveranstaltung "Angewandte Systemwissenschaften 1" in der "Gruppe Plakolb" **relevanten Software** installiert sind.

Python-Pakete und Anaconda-Pakete werden dabei **automatisch installiert**, sollten sie noch nicht vorhanden sein.

Zum Schluss erstellt der ```System Checker``` einen Report über etwaige Fehler bzw die Versionsnummern. Dieser kann über Moodle an die LV-Leitung übermittelt werden.

### Editor

Grundsätzlich ist es in der LV **egal** welcher Text-Editor/welche IDE individuell verwendet wird. Frühere Teilnehmer waren neben dem rudimentären, von [Jupyter](https://jupyter.org/) zur Verfügung gestellten Editor zufrieden mit:

* [Visual Studio Code](https://code.visualstudio.com/)
* [PyCharm](https://www.jetbrains.com/pycharm/)
* [Spyder](https://www.spyder-ide.org/)

Wichtig ist, dass man sich mit der Lösung wohlfühlt :)

### Sinn und Zweck

Der Grund für diesen Check ist, dass gleich zu Beginn der Lehrveranstaltung eine solide Ausgangsbasis auf allen verwendeten Geräten sichergestellt werden soll.

## Ausführen

Um diese Datei auszuführen, klicke auf das &#9193; - Icon in der Toolleiste oben klicken und den Neustart des Kernels bestätigen.

Der Statusbericht befindet sich dann unten.

In [10]:
import os
import sys
import subprocess
import conda
import importlib
import socket
import getpass
import datetime
from functools import reduce
from IPython import display

version = sys.version_info
if version.major == 3 and version.minor > 6:
    run = lambda *cmd: subprocess.run(cmd, capture_output=True)
elif version.major == 3 and version.minor > 4:
    PIPE = subprocess.PIPE
    run = lambda *cmd: subprocess.run(cmd, stdout=PIPE, stderr=PIPE)
else:
    raise ImportError("Please update Python!")

### Helper Functions

In [2]:
def get_version(name, failed=False):
    """Reads the version of a command line tool ``name``.
    
    Parameters
    ----------
    name : str
        The name of the cli tool.
        
    Returns
    -------
    version : str
        ``stdout`` output of a ``name --version`` call.
    """
    version = run(name,'--version')
    if version.stderr:
        install = run('conda','install','-y',name)
        return install
    else:
        return version

def get_import(name, failed=False):
    """Reads the version of a Python library ``name``.
    
    Parameters
    ----------
    name : str
        The name of the library.
        
    Returns
    -------
    version : str
        The output of ``name.__version__``.
    """
    try:
        m = importlib.import_module(name)
        res = m.__version__
    except ImportError as e:
        if not failed:
            run('pip','install',name)
            return check_import(name, failed=True)
        else:
            res = repr(e)
    return res

def show_status(text, version, required=0, best=0):
    """Creates a formatted HTML string graphically
    displaying the version requirements.
    
    Parameters
    ----------
    text : str
        The name of the checked package.
    version : str
        The version string of the package.
    required : int
        Minimum version number required. Default: 0
    best : int
        Recommended version number. Default: 0
        
    Returns
    -------
    html : str
        HTML span tagged version badge.
    """
    version = int(''.join(version.split('.')))
    base = '<span style="background-color:{}; width:90px; margin: 6px; padding: 6px">{}</span>'
    if best > 0 and version >= best:
        color = '#e2fcbf'
    elif version >= required:
        color = '#e6e8a4'
    else:
        color = '#f28d65'
    return base.format(color, text)

def check_version(version, required):
    """Checks a version code against requirements.
    
    Parameters
    ----------
    version : str
        The version string of the package.
    required : int
        Minimum version number required.

    Returns
    -------
    ok : bool
        True if ``version`` meets ``required``.
    """
    version = int(''.join(version.split('.')))
    return version >= required

## Check conda path

The installation path of anaconda cannot contain any space characters (```' '```).

In [3]:
conda_path_ok = conda.CONDA_PACKAGE_ROOT.find(' ') < 0

In [4]:
conda_version = run('conda','--version')

In [5]:
# check version
if int(''.join(conda_version.stdout.decode('utf-8').strip()[-5:].split('.'))) < 400:
    conda_update = run('conda','update','-y','conda')
else:
    conda_update = None

In [6]:
conda = run('conda','info')

In [7]:
conda_env = run('conda','env','list')

In [8]:
# conda_update_all = run('conda','update','-y','--all')

## Check conda installs

Here we check all the packages that should've been installed with ```conda install```

In [9]:
python = get_version('python')
pip = get_version('pip')
git = get_version('git')
jupyter = get_version('jupyter')
run('blablu')

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
%%capture vscode
# VS Code triggers FileNotFoundError when "run('code')"  on Windows
!code --version

## Check python packages

This checks the packages that should've been installed with ```pip install```

In [ ]:
matplotlib = get_import('matplotlib')
numpy = get_import('numpy')
pandas = get_import('pandas')
pip_py = get_import('pip')
pytest = get_import('pytest')
hypothesis = get_import('hypothesis')

In [ ]:
%%capture matplotlib_backend
%matplotlib

# Display Results

In [ ]:
# Prepare version codes and requirements

cot, cov = conda_version.stdout.decode('utf-8').strip().split(' ')[:2]
pyt, pyv = python.stdout.decode('utf-8').strip().split(' ')[:2]
pit, piv = 'pip', pip_py
try:
    jut, juv = [x.split('\n')[1].strip() for x in jupyter.stdout.decode('utf-8').strip().split(':')[:2]]
except IndexError:
    jut, juv = 'Jupyter','0.0.0'
except ValueError:
    jut, juv = 'Jupyter','0.0.0'
    
requirements = [
    (cot, cov, 400, 480),
    (pyt, pyv, 300, 380),
    (pit, piv, 1000, 2000),
    ('matplotlib', matplotlib, 200, 313),
    ('numpy', numpy, 100, 1180),
    ('pandas', pandas, 80, 100),
    ('pytest', pytest, 300, 540),
    ('hypothesis', hypothesis, 400, 580)
]

In [ ]:
status = reduce(lambda a,b: a+b, [show_status(*r) for r in  requirements])
reporting = not any(check_version(v, r) for _, v, r, _ in requirements) and conda_path_ok
if reporting:
    status += show_status('&rArr; Failed, please send the report printed below!', '0', 100)
else:
    status += show_status('&rArr; Success!', '200', 1, 1)
display.HTML(status)

## Report Errors

In [ ]:
def write_report(*contents):
    """Output a report file with ``contents``.
    
    Parameters
    ----------
    *contents : Variable tuple of strings
        The lines that should be printed to the file.
    """
    print('\n'.join(contents))
    with open('syscheck-report-{}-{}-{}.txt'.format(
                socket.gethostname(),
                getpass.getuser(),
                datetime.datetime.now().date().isoformat()
                )
              , 'w') as f:
        
        for line in contents:
            if line:
                f.write(line + '\n')
        else:
            f.write('\n'+hex(hash(reduce(lambda a,b: a+b, contents))))

In [ ]:
if reporting:
    write_report(
        'Report',
        '------',
        datetime.datetime.now().date().isoformat(),
        '\nSystem',
        '------\n',
        '- Platform: {}'.format(sys.platform),
        '- Conda path clean: {}'.format(conda_path_ok),
        '- Conda cli:',
        conda.stdout.decode('utf-8'),
        conda.stderr.decode('utf-8'),
        conda_env.stdout.decode('utf-8'),
        conda_env.stderr.decode('utf-8'),
    #     conda_update_all.stdout.decode('utf-8'),
    #     conda_update_all.stderr.decode('utf-8'),
        '\n- Python cli:',
        python.stdout.decode('utf-8'),
        python.stderr.decode('utf-8'),
        '\n- Pip cli:',
        pip.stdout.decode('utf-8'),
        pip.stderr.decode('utf-8'),
        '\n- Git cli:',
        git.stdout.decode('utf-8'),
        git.stderr.decode('utf-8'),
        '\n-Jupyter cli:',
        jupyter.stdout.decode('utf-8'),
        jupyter.stderr.decode('utf-8'),
        '\n- Visual Studio Code:',
        vscode.stdout,
        vscode.stderr,
        '\nPython modules',
        '--------------\n',
        '- matplotlib: ' + matplotlib,
        '- Backend: ' + matplotlib_backend.stdout,
        matplotlib_backend.stderr,
        '- mumpy: ' + numpy,
        '- pandas: ' + pandas,
        '- pip: ' + pip_py,
        '- pytest: ' + pytest,
        '- hypothesis: ' + hypothesis
        )